# UFoE 실험 — Colab A100 (처음→끝 한 번에)

**Colab A100**에서 셀을 **위에서부터 순서대로** 실행하면: **설치 → 후보 생성(n=100) → 파이프라인(MD) → 결과**까지 한 번에 진행됩니다. `Runtime → Run all` 로 한꺼번에 실행해도 됩니다.

- **n=100** (후보·대조군 각 100개)
- 기본: **mock** 모드 (실제 MD 없이 합성 궤적). A100으로 실제 MD를 돌리려면 파이프라인 셀에서 `run_mode='quick'` 로 바꾼 뒤 다시 실행.
- 저장소는 **Public** 이어야 클론됩니다.

## 1. 설치 (클론 + 의존성)

In [ ]:
import os, subprocess, sys
from pathlib import Path

REPO_NAME = "UFoE-Protein-generation_cursor"
REPO_URL = "https://github.com/plantv-blip/UFoE-Protein-generation_cursor.git"

if not os.path.isdir(REPO_NAME):
    env = os.environ.copy()
    env["GIT_TERMINAL_PROMPT"] = "0"
    r = subprocess.run(["git", "clone", "--depth", "1", REPO_URL], env=env, capture_output=True, text=True, timeout=120)
    if r.returncode != 0:
        print("클론 실패:", r.stderr or r.stdout)
        raise RuntimeError("저장소를 Public으로 설정하거나 토큰으로 클론하세요.")

cwd = Path(os.getcwd())
project_dir = cwd / REPO_NAME / "ufoe_protein_experiment" if REPO_NAME not in str(cwd) else (cwd / "ufoe_protein_experiment" if (cwd / "ufoe_protein_experiment").exists() else cwd)
os.chdir(project_dir)
print("작업 디렉터리:", os.getcwd())

!pip install -q -r requirements.txt
!pip install -q openmm pdbfixer 2>/dev/null || !pip install -q openmm 2>/dev/null || echo "OpenMM/pdbfixer 선택 설치"

import logging
logging.basicConfig(level=logging.INFO, format="%(message)s", stream=sys.stdout)
for _n in ("src.pipeline", "src.folding", "src.generator"):
    logging.getLogger(_n).setLevel(logging.INFO)
print("설치 완료.")

## 2. 후보 생성 (n=100)

In [ ]:
!python3 generate_candidates.py --n 100 --type B --output candidates/ --esmfold mock
print("후보 생성 완료.")

## 3. 파이프라인: 생성 → MD → 결과 (n=100)

In [ ]:
import os, sys
from pathlib import Path

N_TEST = 100  # 후보·대조군 개수
project_dir = Path(".").resolve()
repo_root = project_dir.parent
claude_dir = repo_root / "claude code" / "ufoe_protein_experiment"

if claude_dir.is_dir():
    os.chdir(claude_dir)
    sys.path.insert(0, str(claude_dir))
    from src.pipeline.pipeline import Phase2Pipeline, PipelineConfig
    config = PipelineConfig(
        output_dir="output/colab_n5",
        run_mode="mock",           # 실제 MD: 'quick' (A100 권장)
        esmfold_mode="mock",
        n_samples_2a=N_TEST,
        n_samples_2b=N_TEST,
        n_control_random=N_TEST,
        n_control_inverted=N_TEST,
    )
    pipeline = Phase2Pipeline(config)
    results = pipeline.run_full()
    pipeline.save_results(results)
    Phase2Pipeline.print_summary(results)
else:
    sys.path.insert(0, str(project_dir))
    from src.pipeline.pipeline import run_mock_experiment
    results = run_mock_experiment(n=N_TEST)
    if isinstance(results, dict):
        print("접힘 성공률:", results.get("fold_success_rate"), "| K/M:", results.get("K_over_M"))
    else:
        # PipelineResults (e.g. when claude code was in path)
        r2a = getattr(results, "ufoe_2a_report", None)
        r2b = getattr(results, "elimination_ufoe_2a", None)
        rate = r2a.success_rate if r2a else 0
        km = r2b.self_consistency_ratio if r2b else 0
        print("접힘 성공률:", rate, "| K/M:", km)
print("파이프라인 완료.")

## 4. 결과 시각화

In [ ]:
import matplotlib.pyplot as plt

if hasattr(results, 'ufoe_2a_report') and results.ufoe_2a_report and results.control_random_report:
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))
    ax[0].bar(['UFoE 2a', 'Control'], [results.ufoe_2a_report.success_rate, results.control_random_report.success_rate])
    ax[0].set_ylabel('접힘 성공률')
    if results.elimination_ufoe_2a and results.elimination_control:
        ax[1].bar(['UFoE 2a', 'Control'], [results.elimination_ufoe_2a.self_consistency_ratio, results.elimination_control.self_consistency_ratio])
        ax[1].set_ylabel('K/M 자기일관성')
    plt.tight_layout()
    plt.show()
elif isinstance(results, dict):
    fig, ax = plt.subplots(1, 1, figsize=(5, 4))
    ax.bar(['UFoE'], [results.get('fold_success_rate', 0)])
    ax.set_ylabel('접힘 성공률')
    plt.show()
else:
    print("파이프라인 셀을 먼저 실행하세요.")
print("끝.")